In [ ]:
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Convolution1D,
    MaxPooling1D,
    LSTM,
    Dropout,
    Dense,
    LeakyReLU,
    Activation,
)


def predict_next_day_values(data_path, get_value: str):
    # Load data from JSON file
    with open(data_path) as f:
        data = json.load(f)

    # Extract features and target
    X = np.column_stack((data["o"], data["h"], data["l"], data["c"], data["v"]))
    y = np.array(data[get_value])

    # Normalize data
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

    # Reshape data for model input
    X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

    # Build the model
    model = Sequential()
    model.add(
        Convolution1D(
            input_shape=(X_reshaped.shape[1], 1),
            nb_filter=16,
            filter_length=60,
            border_mode="same",
        )
    )
    model.add(LeakyReLU())
    model.add(Dropout(0.1))
    model.add(Convolution1D(nb_filter=16, filter_length=8, border_mode="same"))
    model.add(LeakyReLU())
    model.add(Dropout(0.1))
    model.add(Convolution1D(nb_filter=8, filter_length=4, border_mode="same"))
    model.add(LeakyReLU())
    model.add(Dropout(0.1))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(150, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(150))
    model.add(LeakyReLU())
    model.add(Dropout(0.1))
    model.add(Dense(128))
    model.add(LeakyReLU())
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(LeakyReLU())
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(LeakyReLU())
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation("tanh"))

    model.compile(optimizer="adam", loss="mean_squared_error")

    # Train the model using all data
    model.fit(X_reshaped, y_scaled, epochs=3, batch_size=32, verbose=0)

    # Predict tomorrow's value
    last_data_point = X_reshaped[-1].reshape(1, X_reshaped.shape[1], 1)
    predicted_value_scaled = model.predict(last_data_point)
    predicted_value = scaler_y.inverse_transform(predicted_value_scaled)

    return predicted_value


# Define data file path
data_path = "hsg.json"

# Define values to predict
values_to_predict = [""]

# Predict for each value
for value in values_to_predict:
    predicted_value = predict_next_day_values(data_path, value)
    print(f"Predicted {value} for tomorrow:", predicted_value)